In [1]:
import os

In [2]:
os.environ["JAVA_HOME"] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_131.jdk/Contents/Home/'
os.environ["PYSPARK_SUBMIT_ARGS"] = 'pyspark-shell'

In [3]:
os.environ["SPARK_HOME"] = '/usr/local/Cellar/apache-spark/2.2.0/libexec'

In [4]:
import sys

sys.path.append(os.environ['SPARK_HOME']+"/python")

In [5]:
sys.path.append(os.environ['SPARK_HOME']+"/python/lib/py4j-0.10.4-src.zip")

In [6]:
import py4j
from pyspark import SparkContext, SparkConf, SQLContext

In [7]:
conf = (SparkConf().setMaster("local[8]")
        .setAppName("ML demo")
        .set("spark.executor.memory", "2g"))

In [8]:
sc = SparkContext(conf=conf)

In [9]:
sqlcontext = SQLContext(sc)

In [10]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionModel, LinearRegressionWithSGD
import numpy as np

In [11]:
from pyspark.ml.classification import GBTClassificationModel
from pyspark.sql.functions import udf
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql import types

In [12]:
data=[
    LabeledPoint(0.0,[0.0]),
    LabeledPoint(1.0,[1.0]),
    LabeledPoint(3.0,[2.0]),
    LabeledPoint(2.0,[3.0])
]
lrm=LinearRegressionWithSGD.train(sc.parallelize(data),iterations=10,initialWeights=np.array([1.0]))
print(lrm.predict(np.array([1.0])))

/usr/local/Cellar/apache-spark/2.2.0/libexec/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


0.928638123469


In [198]:
df = sqlcontext.read.format(
    'com.databricks.spark.csv').options(
    header='true').load('/Users/helenahuddy/Downloads/train.csv')

In [199]:
df.head(10)

[Row(PassengerId='1', Survived='0', Pclass='3', Name='Braund, Mr. Owen Harris', Sex='male', Age='22', SibSp='1', Parch='0', Ticket='A/5 21171', Fare='7.25', Cabin=None, Embarked='S'),
 Row(PassengerId='2', Survived='1', Pclass='1', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38', SibSp='1', Parch='0', Ticket='PC 17599', Fare='71.2833', Cabin='C85', Embarked='C'),
 Row(PassengerId='3', Survived='1', Pclass='3', Name='Heikkinen, Miss. Laina', Sex='female', Age='26', SibSp='0', Parch='0', Ticket='STON/O2. 3101282', Fare='7.925', Cabin=None, Embarked='S'),
 Row(PassengerId='4', Survived='1', Pclass='1', Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35', SibSp='1', Parch='0', Ticket='113803', Fare='53.1', Cabin='C123', Embarked='S'),
 Row(PassengerId='5', Survived='0', Pclass='3', Name='Allen, Mr. William Henry', Sex='male', Age='35', SibSp='0', Parch='0', Ticket='373450', Fare='8.05', Cabin=None, Embarked='S'),
 Row(PassengerId='

In [200]:
def Embarked_transform(x):
    if x != None:
        return x
    else:
        return ''

my_udf = udf(Embarked_transform, types.StringType())
df = df.withColumn('Embarked', my_udf(df['Embarked']))

df.select('Embarked').distinct().collect()

[Row(Embarked='Q'), Row(Embarked='C'), Row(Embarked='S'), Row(Embarked='')]

In [201]:
import re
def article(x):
    return re.split(' ',str(re.findall(", \w+.", x)[0]))[1]

In [202]:
stringIndexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)
encoder = OneHotEncoder(inputCol="EmbarkedIndex", outputCol="EmbarkedVec")
df_t = encoder.transform(indexed)


In [203]:
my_udf_name = udf(article, types.StringType())


df_tt = df_t.withColumn('Name', my_udf_name(df_t['Name']))

In [204]:
df_tt.select('Name').distinct().collect()

[Row(Name='Miss.'),
 Row(Name='Ms.'),
 Row(Name='Sir.'),
 Row(Name='Rev.'),
 Row(Name='Mr.'),
 Row(Name='Major.'),
 Row(Name='Mrs.'),
 Row(Name='Dr.'),
 Row(Name='Col.'),
 Row(Name='Master.'),
 Row(Name='Mlle.'),
 Row(Name='Jonkheer.'),
 Row(Name='Mme.'),
 Row(Name='the'),
 Row(Name='Lady.'),
 Row(Name='Capt.'),
 Row(Name='Don.')]

In [205]:
stringIndexer = StringIndexer(inputCol="Name", outputCol="NameIndex")
model = stringIndexer.fit(df_tt)
indexed = model.transform(df_tt)
encoder = OneHotEncoder(inputCol="NameIndex", outputCol="NameVec")
df_t = encoder.transform(indexed)

In [210]:
def cat_cab(x):
    if x!=None:
        return x[0]
    else:
        return 'N'

In [211]:
my_udf_cab = udf(cat_cab, types.StringType())

df_tt = df_t.withColumn('Num_Cab', my_udf_cab(df_t['Cabin']))

In [212]:
df_tt

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string, EmbarkedIndex: double, EmbarkedVec: vector, NameIndex: double, NameVec: vector, Num_Cab: string]

In [213]:
stringIndexer = StringIndexer(inputCol="Num_Cab", outputCol="Num_CabIndex")
model = stringIndexer.fit(df_tt)
indexed = model.transform(df_tt)
encoder = OneHotEncoder(inputCol="Num_CabIndex", outputCol="Num_CabVec")
df_t = encoder.transform(indexed)

In [214]:
df_t

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string, EmbarkedIndex: double, EmbarkedVec: vector, NameIndex: double, NameVec: vector, Num_Cab: string, Num_CabIndex: double, Num_CabVec: vector]

In [216]:
df_t.head(2)

[Row(PassengerId='1', Survived='0', Pclass='3', Name='Mr.', Sex='male', Age='22', SibSp='1', Parch='0', Ticket='A/5 21171', Fare='7.25', Cabin=None, Embarked='S', EmbarkedIndex=0.0, EmbarkedVec=SparseVector(3, {0: 1.0}), NameIndex=0.0, NameVec=SparseVector(16, {0: 1.0}), Num_Cab='N', Num_CabIndex=0.0, Num_CabVec=SparseVector(8, {0: 1.0})),
 Row(PassengerId='2', Survived='1', Pclass='1', Name='Mrs.', Sex='female', Age='38', SibSp='1', Parch='0', Ticket='PC 17599', Fare='71.2833', Cabin='C85', Embarked='C', EmbarkedIndex=1.0, EmbarkedVec=SparseVector(3, {1: 1.0}), NameIndex=2.0, NameVec=SparseVector(16, {2: 1.0}), Num_Cab='C', Num_CabIndex=1.0, Num_CabVec=SparseVector(8, {1: 1.0}))]

In [217]:
def parse_age(str_age):
    try:
        return float(str_age)
    except:
        return -1

In [218]:
def transf(r):
    return LabeledPoint(
        int(r.Survived),
        [
            int(r.Pclass),
            r.Sex == 'male',
            float(r.Fare),
            int(r.SibSp),
            int(r.Parch),
            parse_age(r.Age),
            r.Cabin==None,
            int(r.SibSp) + int(r.Parch),
            int(r.SibSp) + int(r.Parch)==0
            
        ] + list(r.EmbarkedVec.toArray())
        + list(r.NameVec.toArray())
        + list(r.Num_CabVec.toArray())


    )

In [219]:
data = df_t.rdd.map(transf)

In [220]:
train, test = data.randomSplit([0.7, 0.3])

In [221]:
train.cache()

PythonRDD[1241] at RDD at PythonRDD.scala:48

In [226]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.classification import  SVMWithSGD
from pyspark.mllib.tree import GradientBoostedTrees
from pyspark.mllib.tree import RandomForest, RandomForestModel






rfc = RandomForest.trainClassifier(train, numClasses=2,
                             categoricalFeaturesInfo={},
                            numTrees=100)
gbt = GradientBoostedTrees.trainClassifier(train, {}, numIterations=10)
logreg = LogisticRegressionWithLBFGS.train(train, iterations=10, numClasses=2)
svm = SVMWithSGD.train(train, iterations=10)
lrm = LogisticRegressionWithSGD.train(train, iterations=10)

/usr/local/Cellar/apache-spark/2.2.0/libexec/python/pyspark/mllib/classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


In [43]:
def acc(m, test):
    values = test.map(lambda x: x.features)
    yhat = m.predict(values)
    y = test.map(lambda x: x.label)
    comp = yhat.zip(y)
    errors = comp.map(lambda x: abs(x[0]-x[1]))
    return 1-errors.sum()/errors.count()

In [224]:
acc(rfc, test)

0.8202247191011236

In [228]:
acc(logreg,test)

0.8202247191011236

In [225]:
acc(gbt,test)

0.8239700374531835

In [227]:
acc(svm,test)

0.6104868913857677

In [238]:
from pyspark.mllib.evaluation import MulticlassMetrics
def f(m, test, label=1.0):
    values = test.map(lambda x: x.features)
    yhat = m.predict(values)
    y = test.map(lambda x: x.label)
    comp = yhat.zip(y)
    metrics = MulticlassMetrics(predictionAndLabels)
    return metrics.fMeasure(label=label)

In [239]:
f(logreg,test)

0.7551020408163265

In [237]:
f(svm,test)

0.49019607843137253

In [ ]:
# добавить 5 новых фичей
# 3 фичи высчитываются из имеющихся
# хотя бы одна использует udf

# попробовать 3 новых модели

# f1 меру